In [1]:
import random
import csv
import sys
import math
import matplotlib.pyplot as plt
from collections import Counter


In [2]:
def csv_file_to_ordered_data(csv_file_name):
    data = csv_file_to_list(csv_file_name)
    return order_csv_data(data)

def is_complete(data_item, pos):
    return data_item[pos] != '?'

def order_csv_data(csv_data):
    # The first row in the CSV file is the heading of the data table.
    heading = csv_data.pop(0)
    complete_data = []
    incomplete_data = []

    # Let enquired_column be the column of the variable which
    # conditional probability should be calculated. Here set that
    # column to be the last one.
    enquired_column = len(heading) - 1
    
    # Divide the data into the complete and the incomplete data.
    # An incomplete row is the one that has a question mark in the
    # enquired_column. The question mark will be replaced by the
    # calculated Baysian probabilities from the complete data.
    for data_item in csv_data:
        if is_complete(data_item, enquired_column):
            complete_data.append(data_item)
        else:
            incomplete_data.append(data_item)
    return (heading, complete_data, incomplete_data, enquired_column)

def csv_file_to_list(csv_file_name):
    with open(csv_file_name, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
    return data


In [3]:
(heading, complete_data, incomplete_data, enquired_column) = csv_file_to_ordered_data("swim.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'swim.csv'

In [4]:
complete_data

NameError: name 'complete_data' is not defined

In [5]:
incomplete_data

NameError: name 'incomplete_data' is not defined

In [6]:
def split_data_by_col(data, col):
    data_groups = {}
    for data_item in data:
        if data_groups.get(data_item[col]) is None:
            data_groups[data_item[col]] = []
        data_groups[data_item[col]].append(data_item)
    return data_groups


In [7]:
res = split_data_by_col(complete_data, 1)

NameError: name 'complete_data' is not defined

In [8]:
res


NameError: name 'res' is not defined

In [9]:
res = split_data_by_col(complete_data, 0)

NameError: name 'complete_data' is not defined

In [10]:
res

NameError: name 'res' is not defined

In [11]:
def col_information_gain(complete_data, col, enquired_column):
    data_groups = split_data_by_col(complete_data, col)
    information_gain = entropy(complete_data, enquired_column)
    for _, data_group in data_groups.items():
        information_gain -= (float(len(data_group)) / len(complete_data)
                             ) * entropy(data_group, enquired_column)
    return information_gain


In [12]:
def entropy(data, enquired_column):
    value_counts = {}
    for data_item in data:
        if value_counts.get(data_item[enquired_column]) is None:
            value_counts[data_item[enquired_column]] = 0
        value_counts[data_item[enquired_column]] += 1
    entropy = 0
    for _, count in value_counts.items():
        probability = float(count) / len(data)
        entropy -= probability * math.log(probability, 2)
    return entropy


In [13]:
col_information_gain(complete_data, 0, enquired_column)

NameError: name 'complete_data' is not defined

In [14]:
col_information_gain(complete_data, 1, enquired_column)

NameError: name 'complete_data' is not defined

In [15]:
# we need a storage structure for the tree

class TreeNode:

    def __init__(self, var=None, val=None):
        self.children = []
        self.var = var
        self.val = val

    def add_child(self, child):
        self.children.append(child)

    def get_children(self):
        return self.children

    def get_var(self):
        return self.var

    def get_val(self):
        return self.val

    def is_root(self):
        return self.var is None and self.val is None

    def is_leaf(self):
        return len(self.children) == 0

    def name(self):
        if self.is_root():
            return "[root]"
        return "[" + self.var + "=" + self.val + "]"


In [16]:
# m is the number of the classifying variables that should be at most
# considered at each node. m needed only for a random forest. -> later
# for decision tree all variables are used -> len(heading)

def construct_general_tree(heading, complete_data, enquired_column, m):
    available_columns = []
    for col in range(0, len(heading)):
        if col != enquired_column:
            available_columns.append(col)
    tree = TreeNode()
    add_children_to_node(tree, heading, complete_data,
                         available_columns, enquired_column, m)
    return tree


# as described above ...

def constuct_decision_tree(heading, complete_data, enquired_column):
    return construct_general_tree(heading, complete_data,
                                  enquired_column, len(heading))



In [17]:
def add_children_to_node(node, heading, complete_data, available_columns, enquired_column, m):
    if len(available_columns) == 0:
        # no more variables to split - just add leaf
        add_leaf(node, heading, complete_data, enquired_column)
        return -1

    # choose the next column for splitting
    selected_col = select_col(
        heading, complete_data, available_columns,
        enquired_column, m)
    for i in range(0, len(available_columns)):
        if available_columns[i] == selected_col:
            available_columns.pop(i)
            break

    # now calculate 
    data_groups = split_data_by_col(complete_data, selected_col)
    if (len(data_groups.items()) == 1):
        # there is only one decision solution left - just add a leaf
        add_leaf(node, heading, complete_data, enquired_column)
        return -1

    # do the recursion: add node and then add remaining children
    
    for child_group, child_data in data_groups.items():
        child = TreeNode(heading[selected_col], child_group)
        add_children_to_node(child, heading, child_data, list(
            available_columns), enquired_column, m)
        node.add_child(child)


In [18]:
# Selects an available column/attribute with the highest information gain

def select_col(heading, complete_data, available_columns, enquired_column, m):
    # Consider only a subset of the available columns of size m.

    if len(available_columns) < m:
        sample_columns = available_columns
    else:
        sample_columns = random.sample(available_columns, m)

    selected_col = -1
    selected_col_information_gain = -1
    for col in sample_columns:
        current_information_gain = col_information_gain(
            complete_data, col, enquired_column)

        # print len(complete_data),col,current_information_gain
        if current_information_gain > selected_col_information_gain:
            selected_col = col
            selected_col_information_gain = current_information_gain

    return selected_col


In [19]:
def add_leaf(node, heading, complete_data, enquired_column):
    leaf_node = TreeNode(heading[enquired_column],
                         complete_data[0][enquired_column])
    node.add_child(leaf_node)


In [20]:
tree = constuct_decision_tree(heading, complete_data, enquired_column)

NameError: name 'heading' is not defined

In [21]:
tree


NameError: name 'tree' is not defined

In [22]:
# A simple textual output of a tree without the visualization.
def display_tree(tree):
    print('***Tree structure***')
    display_node(tree)
    sys.stdout.flush()

# A simple textual output of a node in a tree.
def display_node(node):
    if node.is_leaf():
        print('The node ' + node.name() + ' is a leaf node.')
        return
    sys.stdout.write('The node ' + node.name() + ' has children: ')
    for child in node.get_children():
        sys.stdout.write(child.name() + ' ')
    print('')
    for child in node.get_children():
        display_node(child)


In [23]:
display_tree(tree)

NameError: name 'tree' is not defined

In [24]:
def classify_by_tree(tree, heading, enquired_column, feature):
    var_to_index = {}
    for index in range(0, len(heading)):
        var_to_index[heading[index]] = index
    return classify_by_tree_recurse(tree, var_to_index, feature)


def classify_by_tree_recurse(tree, var_to_index, feature):
    if tree.is_leaf():
        return tree.get_val()
    else:
        for child in tree.get_children():
            if child.is_leaf():
                return child.get_val()
            if child.get_val() == feature[
               var_to_index.get(child.get_var())]:
                return classify_by_tree_recurse(
                       child, var_to_index, feature)
        return None


In [25]:
classify_by_tree(tree, heading, enquired_column, ['Small', 'Cold'])

NameError: name 'tree' is not defined

In [26]:
classify_by_tree(tree, heading, enquired_column, ['Good', 'Warm'])

NameError: name 'tree' is not defined

In [27]:
(heading, complete_data, incomplete_data, enquired_column) = csv_file_to_ordered_data("chess.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'chess.csv'

In [28]:
complete_data

NameError: name 'complete_data' is not defined

In [29]:
incomplete_data

NameError: name 'incomplete_data' is not defined

In [30]:
# here it is time for the next exercise ...

In [31]:
# random forest

(heading, complete_data, incomplete_data, enquired_column) = csv_file_to_ordered_data("swim.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'swim.csv'

In [32]:
complete_data

NameError: name 'complete_data' is not defined

In [33]:
dataset1 = [['None', 'Warm', 'No'], ['None', 'Warm', 'No'], ['Small', 'Cold', 'No'],
  ['Good', 'Cold', 'No'], ['Good', 'Cold', 'No'], ['Good', 'Cold', 'No']]

In [34]:
dataset1

[['None', 'Warm', 'No'],
 ['None', 'Warm', 'No'],
 ['Small', 'Cold', 'No'],
 ['Good', 'Cold', 'No'],
 ['Good', 'Cold', 'No'],
 ['Good', 'Cold', 'No']]

In [35]:
tree1 = constuct_decision_tree(heading, dataset1, enquired_column)

NameError: name 'heading' is not defined

In [36]:
display_tree(tree1)

NameError: name 'tree1' is not defined

In [37]:
dataset2 = [['Good', 'Warm', 'Yes'], ['None', 'Warm', 'No'], ['Good', 'Cold', 'No'],
           ['None', 'Cold', 'No'], ['None', 'Warm', 'No'],['Small', 'Warm', 'No'] ]
  


In [38]:
dataset2

[['Good', 'Warm', 'Yes'],
 ['None', 'Warm', 'No'],
 ['Good', 'Cold', 'No'],
 ['None', 'Cold', 'No'],
 ['None', 'Warm', 'No'],
 ['Small', 'Warm', 'No']]

In [39]:
tree2 = constuct_decision_tree(heading, dataset2, enquired_column)

NameError: name 'heading' is not defined

In [40]:
display_tree(tree2)

NameError: name 'tree2' is not defined

In [41]:
classify_by_tree(tree1, heading, enquired_column, ['Good', 'Cold'])

NameError: name 'tree1' is not defined

In [42]:
classify_by_tree(tree2, heading, enquired_column, ['Good', 'Cold'])

NameError: name 'tree2' is not defined

In [43]:
#create random sample of features
def sample_with_replacement(population, size):
    sample = []
    for i in range(0, size):
        sample.append(population[random.randint(0, len(population) - 1)])
    return sample


In [44]:
sample_with_replacement(complete_data, 6)

NameError: name 'complete_data' is not defined

In [45]:
sample_with_replacement(complete_data, 6)

NameError: name 'complete_data' is not defined

In [46]:
def construct_random_decision_tree(heading, complete_data, enquired_column, m):
    sample = sample_with_replacement(complete_data, len(complete_data))
    return construct_general_tree(heading, sample, enquired_column, m)


In [47]:
def construct_random_forest(heading, complete_data, enquired_column, m, tree_count):
    random_forest = []
    for i in range(0, tree_count):
        tree = construct_random_decision_tree(heading, complete_data, enquired_column, m)
        random_forest.append(tree)
    return random_forest


In [48]:
def display_classification_for_feature(random_forest, heading, enquired_column, feature):
    classification = {}
    for i in range(0, len(random_forest)):
        group = classify_by_tree(random_forest[i], heading, enquired_column, feature)
        dic_inc(classification, group)

    return dic_key_max_count(classification)


In [49]:
# we need the dic functions - we have used them already ...
# Increments integer values in a dictionary.

def dic_inc(dic, key):
    if key is None:
        pass
    if dic.get(key, None) is None:
        dic[key] = 1
    else:
        dic[key] = dic[key] + 1

In [50]:
# Returns a dictionary key with the maximal count.
def dic_key_max_count(dic):
    key_max_count = None
    for key, count in dic.items():
        if key is not None and (key_max_count
           is None or count > dic[key_max_count]):
            key_max_count = key
    return key_max_count


In [51]:
forest = construct_random_forest(heading, complete_data, enquired_column, len(heading), 6)

NameError: name 'heading' is not defined

In [52]:
forest

NameError: name 'forest' is not defined

In [53]:
display_classification_for_feature(forest, heading, enquired_column, ['Good','Cold'])

NameError: name 'forest' is not defined

In [54]:
display_classification_for_feature(forest, heading, enquired_column, ['Good','Warm'])

NameError: name 'forest' is not defined

In [55]:
# for exercise 8 you need a function to output the trees of the forest
(heading, complete_data, incomplete_data, enquired_column) = csv_file_to_ordered_data("chess_with_seasons.csv")
chess_features = incomplete_data[0][:3]
chess_forest = construct_random_forest(heading,complete_data,enquired_column,len(heading),10)


In [56]:
display_classification_for_feature(chess_forest,heading,enquired_column,chess_features)

'No'